<a href="https://colab.research.google.com/github/seanvw/TR_Trade_1/blob/main/Share_dealing_v8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**To deepseek-r1-distill-llama-70b with Groq**

Write a python application to make calculations of profit/loss and APR for each Share held in a Portfolio.

Include a demonstation of it's usage.

You can use the current share price for a given ticker symbol using yfinance.

A share can be bought and sold within a day or over a longer period, months or years. It is very important that a share can be bought and sold within the same day, therefore the calaculation of days should not be an integer, but rather a floating point number.

When a share is bought, the date and time is captured to the minute.

A share that is already held may be bought again at a later data (almost certainly for a different price). The number of shares held should be adjusted and the bought price recalculated appropriately.  

When a share is sold, the date and time is captured to the minute.

The application should support a patial sale of a share holding e.g. when 10 shares are held and only 5 are sold.

Foreach share held within the portfolio where the sale was realized the application should calculate the realised profit/loss, the APR, together with its bought and sold datetimes and the days elapsed between purchase and sale.  

Foreach share held within the portfolio where the sale was not realized (i.e. unrealized) the application should calculate the profit/loss and APR based upon the current share price from yfinance as if the share was sold at that point in time (this can be thought of as the effective APR)

A final report should bring together realized and unrealized share dealings in a summary that calculates the grand total APR and profit/loss.

The demonstation of usage should give examples of realised share dealings held for 0.5 day, 1 day, 5 days and 6 months respectively

The demonstation of usage should give an example of the same share bought twice.

The demonstation of usage should give an example of a partial share sale.

The demonstation of usage should give examples of unrealised share purchases

Respond with the code only! Nothing else!

--------------------

Okay, after adding a dunder __str__ to Portfolio I can see how this code is working or rather not working.

The sale of shares isn't working really, because, the purchase of new shares should change the holding price.

The transaction class is only a statement of purchasing...

The realized transaction report does produce anything even though we sold ...

In [ ]:
import yfinance as yf
from datetime import datetime
import pprint

class Transaction:
    def __init__(self, buy_price, quantity, buy_datetime):
        self.buy_price = buy_price
        self.quantity = quantity
        self.buy_datetime = buy_datetime

class Share:
    def __init__(self, ticker):
        self.ticker = ticker
        self.transactions = []

    def add_transaction(self, buy_price, quantity, buy_datetime):
        self.transactions.append(Transaction(buy_price, quantity, buy_datetime))

    def sell_transaction(self, sell_price, sell_datetime, quantity):
        sold = 0
        for i in range(len(self.transactions)):
            if self.transactions[i].quantity > 0:
                available = self.transactions[i].quantity
                to_sell = min(available, quantity - sold)
                if to_sell > 0:
                    self.transactions[i].quantity -= to_sell
                    sold += to_sell
                    yield {
                        'buy_price': self.transactions[i].buy_price,
                        'buy_datetime': self.transactions[i].buy_datetime,
                        'sell_price': sell_price,
                        'sell_datetime': sell_datetime,
                        'quantity': to_sell
                    }
            if sold >= quantity:
                break

    def calculate_apr(self, current_price=None):
        current_price = current_price or yf.Ticker(self.ticker).info['currentPrice']
        total_profit = 0
        total_cost = 0
        for transaction in self.transactions:
            if transaction.quantity > 0:
                days = (datetime.now() - transaction.buy_datetime).total_seconds() / (24 * 3600)
                profit = (current_price - transaction.buy_price) * transaction.quantity
                total_profit += profit
                total_cost += transaction.buy_price * transaction.quantity
        if total_cost == 0:
            return 0
        apr = (total_profit / total_cost) / (days / 365) * 100
        return apr

class Portfolio:
    def __init__(self):
        self.shares = {}

    def __str__(self) -> str:
        for ticker, share in self.shares.items():
            print(f"{ticker}: ")
            for transaction in share.transactions:
                print(f"    {transaction.buy_price} {transaction.quantity} {transaction.buy_datetime}")
        return ""


    def add_share(self, ticker, buy_price, quantity, buy_datetime):
        if ticker not in self.shares:
            self.shares[ticker] = Share(ticker)
        self.shares[ticker].add_transaction(buy_price, quantity, buy_datetime)

    def sell_share(self, ticker, sell_price, sell_datetime, quantity):
        if ticker in self.shares:
            # is missing ticker?!
            return list(self.shares[ticker].sell_transaction(sell_price, sell_datetime, quantity))

    def generate_report(self):
        realized = []
        unrealized = []
        total_profit = 0
        total_apr = 0

        for ticker, share in self.shares.items():
            current_price = yf.Ticker(ticker).info['currentPrice']
            for transaction in share.transactions:
                if transaction.quantity > 0:
                    days = (datetime.now() - transaction.buy_datetime).total_seconds() / (24 * 3600)
                    profit = (current_price - transaction.buy_price) * transaction.quantity
                    apr = share.calculate_apr(current_price)
                    unrealized.append({
                        'ticker': ticker,
                        'buy_datetime': transaction.buy_datetime.strftime('%Y-%m-%d %H:%M'),
                        'current_price': round(current_price, 2),
                        'unrealized_profit': round(profit, 2),
                        'apr': round(apr, 2)
                    })
                    total_profit += profit
                    total_apr += apr

        summary = {
            'total_profit': round(total_profit, 2),
            'average_apr': round(total_apr / len(unrealized) if unrealized else 0, 2)
        }

        return realized, unrealized, summary


#portfolio.add_share('MSFT', 300.0, 5, datetime(2023, 10, 2, 10, 0))
#portfolio.add_share('GOOG', 2500.0, 2, datetime(2023, 4, 1, 10, 0))
#portfolio.add_share('AMZN', 1800.0, 3, datetime(2023, 10, 1, 10, 0))


# Demonstration of usage
portfolio = Portfolio()

# Example of buying and selling shares
portfolio.add_share('AAPL', 150.0, 10, datetime(2023, 10, 1, 9, 30))
print(portfolio)

portfolio.add_share('AAPL', 155.0, 5, datetime(2023, 10, 1, 10, 0))
print(portfolio)

sold = portfolio.sell_share(ticker, 160.0, datetime(2023, 10, 1, 16, 0), 11.5)

print(portfolio)

print("-" * 30)
print("Sold")
for sale in sold:
    print(f"""
    Buy Time: {sale['buy_datetime'].strftime('%Y-%m-%d %H:%M')}
    Sell Time: {sale['sell_datetime'].strftime('%Y-%m-%d %H:%M')}
    Quantity: {sale['quantity']}
    Buy Price: {sale['buy_price']}
    Sell Price: {sale['sell_price']}
    """)
print("-" * 30)

print(portfolio)

# Generate reports
realized, unrealized, summary = portfolio.generate_report()

# Print reports
print("Realized Transactions:")
for transaction in realized:
    print(f"""
    Ticker: {transaction['ticker']}
    Buy Time: {transaction['buy_datetime']}
    Sell Time: {transaction['sell_datetime']}
    Quantity: {transaction['quantity']}
    Buy Price: {transaction['buy_price']}
    Sell Price: {transaction['sell_price']}
    Profit: {transaction['profit']}
    APR: {transaction['apr']}%
    """)

print("\nUnrealized Holdings:")
for holding in unrealized:
    print(f"""
    Ticker: {holding['ticker']}
    Buy Time: {holding['buy_datetime']}
    Current Price: {holding['current_price']}
    Unrealized Profit: {holding['unrealized_profit']}
    APR: {holding['apr']}%
    """)

print("\nPortfolio Summary:")
print(f"Total Profit: {summary['total_profit']}")
print(f"Average APR: {summary['average_apr']}%")

AAPL: 
    150.0 10 2023-10-01 09:30:00

AAPL: 
    150.0 10 2023-10-01 09:30:00
    155.0 5 2023-10-01 10:00:00

AAPL: 
    150.0 0 2023-10-01 09:30:00
    155.0 3.5 2023-10-01 10:00:00

------------------------------
Sold

    Buy Time: 2023-10-01 09:30
    Sell Time: 2023-10-01 16:00
    Quantity: 10
    Buy Price: 150.0
    Sell Price: 160.0
    

    Buy Time: 2023-10-01 10:00
    Sell Time: 2023-10-01 16:00
    Quantity: 1.5
    Buy Price: 155.0
    Sell Price: 160.0
    
------------------------------
AAPL: 
    150.0 0 2023-10-01 09:30:00
    155.0 3.5 2023-10-01 10:00:00

Realized Transactions:

Unrealized Holdings:

    Ticker: AAPL
    Buy Time: 2023-10-01 10:00
    Current Price: 239.54
    Unrealized Profit: 295.89
    APR: 38.04%
    

Portfolio Summary:
Total Profit: 295.89
Average APR: 38.04%
